In [1]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Text Processing

    Read each sentence from training file and remove the tabs and extract id and tweet seperately. 

In [2]:
file = "data/train_tweets.txt"
temp = []
with open(file, 'r',encoding="utf-8") as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row) 

In [3]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
temp = []
elem = []

In [4]:
tw.head()

,User,Tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


In [ ]:
#data pre-processing
tw['num_of_words'] = tw["Tweet"].str.split().apply(len)
tw.drop(tw[tw.num_of_words<4].index, inplace=True)
tw["Tweet"].replace(r"http\S+", "URL", regex=True,inplace=True)
tw["Tweet"].replace(r"@\S+", "REF", regex=True ,inplace=True)
tw["Tweet"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
tw["Tweet"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
tw["Tweet"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
tw["Tweet"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(tw)

In [ ]:
cnt_user = tw['User'].value_counts()
cnt_user.describe()

In [ ]:
filtered_users=[]
Users=tw.User.unique().tolist()

for a in tw.User.unique():
    if cnt_user[a]>=50:
      filtered_users.append(a)
  

In [ ]:
import random
from sklearn.model_selection import train_test_split
#1000 random sample rows for each author
df_new=pd.DataFrame()
twts_train=pd.DataFrame()
twts_test=pd.DataFrame()
author_train=pd.DataFrame()
author_test=pd.DataFrame()

for a in filtered_users:
    rows = random.sample(list(tw[tw['User']==a].index),50)
    df_temp = tw.loc[rows]
    df_new=df_new.append(df_temp,ignore_index=True) 
    '''   
    X_train, X_test, Y_train, Y_test = train_test_split(df_temp.loc[:,['Tweet']], df_temp.loc[:,['User']], test_size=0.2, random_state=42)
    twts_train=twts_train.append(X_train, verify_integrity=False)
    twts_test=twts_test.append(X_test, verify_integrity=False)
    author_train=author_train.append(Y_train, verify_integrity=False)
    author_test=author_test.append(Y_test, verify_integrity=False)
    
    '''

In [ ]:
df_new=tw
X = df_new.Tweet
y = df_new.User
twts_train, twts_test, author_train, author_test = train_test_split(X,y, random_state=42)

In [ ]:
print (len(twts_train),len(author_train))

In [ ]:
print(len(twts_test),len(author_test))

In [ ]:
df_new.head()

In [ ]:
cnt_user = df_new['User'].value_counts()
cnt_user.describe()

In [ ]:
df_new.tail()

In [ ]:
df_new.shape

In [ ]:
df_new.shape

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tknzr = TweetTokenizer()
    text_processed=tknzr.tokenize(text)
    
    
    
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [ ]:
X_train=twts_train
X_test=twts_test
y_train=author_train
y_test=author_test
print(y_train.shape[0])

In [ ]:
from scipy.sparse import hstack

lr = LogisticRegression()
nb = MultinomialNB()
svm = LinearSVC()

def test_model(w_vect,c_vect, model):
    w_vect.fit(pd.Series(df_new['Tweet']))
    c_vect.fit(pd.Series(df_new['Tweet']))         
    X_train_dtm_w = w_vect.transform(X_train)
    X_train_dtm_c = c_vect.transform(X_train)
    X_train_dtm = hstack([X_train_dtm_c, X_train_dtm_w])
    
    print('Features: ', X_train_dtm.shape[1])
    print(f'Rows: {X_train_dtm.shape[0]}')
    X_test_dtm_w = w_vect.transform(X_test)
    X_test_dtm_c = c_vect.transform(X_test)
    X_test_dtm = hstack([X_test_dtm_c, X_test_dtm_w])

    
    
    if model == 'LR':
        lr.fit(X_train_dtm, y_train)
        y_pred_class = lr.predict(X_test_dtm)
        algorithm = 'Logistic Regression'
    if model == 'MNB':
        nb.fit(X_train_dtm, y_train)
        y_pred_class = nb.predict(X_test_dtm)
        algorithm = 'Multinomial Naive Bayes'
    if model == 'SVC':
        svm.fit(X_train_dtm, y_train)
        y_pred_class = svm.predict(X_test_dtm)
        algorithm = 'Linear SVC'
        
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
    print(algorithm)

In [ ]:
word_vectorizer = CountVectorizer(analyzer='word',tokenizer= text_process, ngram_range = (1,2),min_df = 1, max_features = 150000)
char_vectorizer = CountVectorizer(analyzer='char', stop_words='english', ngram_range=(2,4),max_features=50000)






In [ ]:
%%time
test_model(word_vectorizer, char_vectorizer, 'LR')

In [ ]:
def prepare_test_data(w_vect,c_vect):
    file1 = "data/test_tweets_unlabeled.txt"
    with open(file1, 'r',encoding="utf-8") as data:
        temp = [line for line in data]    
    unlabel = pd.Series(temp,name="value")
    data=unlabel.to_frame()
    data['num_of_words'] = data["value"].str.split().apply(len)
    tw.drop(tw[tw.num_of_words<4].index, inplace=True)
    data["value"].replace(r"http\S+", "URL", regex=True,inplace=True)
    data["value"].replace(r"@\S+", "REF", regex=True ,inplace=True)
    data["value"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
    data["value"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
    data["value"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
    data["value"].replace(r"\d+", "NUM", regex=True,inplace=True)
    #print(unlabel)
    unlabel_dtm_w = w_vect.transform(data["value"])
    unlabel_dtm_c = c_vect.transform(data["value"])
    unlabel_dtm = hstack([unlabel_dtm_c, unlabel_dtm_w])
    return unlabel_dtm
    
def submission_file(data):
    import csv
    with open('predicted.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(['Id','Predicted'])
        for count,predicted in enumerate(data):
            writer.writerow([count+1,predicted])

In [ ]:
unlabel_dtm = prepare_test_data(word_vectorizer, char_vectorizer) 
unlabel_pred = svm.predict(unlabel_dtm)
submission_file(unlabel_pred)

In [ ]:
model_filename = "linearsvc_model_cw_ngram.pkl"
joblib.dump(svm, model_filename)